In [1]:
import nextgenfilter as ngf
import nextgenanalyze as nga
import ngsCSVAnalyze as csva

import yaml
import cPickle as pickle
import sys, os

import pandas as pd

import logging
import time
import progressbar

import matplotlib.pyplot as plt

%matplotlib inline

# Load YAML file, set up things

In [9]:
folder = r"C:\Users\Ted\Google Drive\To File - Research Docs\Screening\nextgenAnalysis\Data\2015_12_01_AMD"
yfname = folder+'\\'+'samples_all.yaml'

with open(yfname) as expt_f:
    expt_yaml = yaml.load(expt_f)